<a href="https://colab.research.google.com/github/sunnysavita10/Indepth-GENAI/blob/main/RAG_With_Knowledge_graph(Neo4j).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **langchain-core**

contains simple, core abstractions that have emerged as a standard, as well as LangChain Expression Language as a way to compose these components together. This package is now at version 0.1 and all breaking changes will be accompanied by a minor version bump.

## **langchain-community**
contains all third party integrations. We will work with partners on splitting key integrations out into standalone packages over the next month.

## **langchain**
contains higher-level and use-case specific chains, agents, and retrieval algorithms that are at the core of your application's cognitive architecture. We are targeting a launch of a stable 0.1 release for langchain in early January.#

In [2]:
# Import statements for various modules and classes needed for the script.
import os  # Basic operating system functionality
from langchain_core.runnables import (  # Runnable classes for different types of operations
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate  # Template for chat prompts
from langchain_core.prompts.prompt import PromptTemplate  # Template for prompts
from typing import Tuple, List, Optional  # Type hints for function signatures
from langchain_core.messages import AIMessage, HumanMessage  # Message classes for AI and human interaction
from langchain_core.output_parsers import StrOutputParser  # Parser for string outputs
from langchain_core.runnables import ConfigurableField  # Configurable fields for operations
from yfiles_jupyter_graphs import GraphWidget  # Widget for displaying graphs in Jupyter
from neo4j import GraphDatabase  # Neo4j graph database connection
from langchain_community.vectorstores import Neo4jVector  # Vector store using Neo4j
from langchain_community.graphs import Neo4jGraph  # Graph operations using Neo4j
from langchain.text_splitter import TokenTextSplitter  # Text splitter based on tokens
from langchain_groq import ChatGroq  # Query language for chatting
from langchain_experimental.graph_transformers import LLMGraphTransformer  # Experimental graph transformer
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
import os
from dotenv import load_dotenv

In [3]:
# Load environment variables from the system
NEO4J_URI = os.environ.get("NEO4J_URI")  # Retrieves the Neo4j URI from environment variables
NEO4J_USERNAME = os.environ.get("NEO4J_USERNAME")  # Retrieves the Neo4j username
NEO4J_PASSWORD = os.environ.get("NEO4J_PASSWORD")  # Retrieves the Neo4j password
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')  # Retrieves the Groq API key

# Setting environment variables for Neo4j and GROQ API in the current session
os.environ["NEO4J_URI"] = NEO4J_URI  # Stores the Neo4j URI in the environment variables
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME  # Stores the Neo4j username
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD  # Stores the Neo4j password
os.environ['Groq_api_key'] = GROQ_API_KEY  # Stores the Groq API key in environment variables


In [41]:
# Establish a connection to the Neo4j graph database
graph = Neo4jGraph(
    url=NEO4J_URI,        # Neo4j database URI
    username=NEO4J_USERNAME,  # Username for authentication
    password=NEO4J_PASSWORD,  # Password for authentication
)

In [4]:
from langchain_community.document_loaders import CSVLoader

# Define the path to your CSV file
file_path = r"F:\GraphRAG-LLM\data\customer_support_tickets.csv"

# Initialize the CSVLoader
loader = CSVLoader(file_path=file_path)

# Load the documents
raw_documents = loader.load()

# Display content and metadata of the first document
print(raw_documents[0].page_content[:10])
print(raw_documents[0].metadata)


Ticket ID:
{'source': 'F:\\GraphRAG-LLM\\data\\customer_support_tickets.csv', 'row': 0}


In [5]:
len(raw_documents)

8469

In [6]:
raw_documents[:3]

[Document(metadata={'source': 'F:\\GraphRAG-LLM\\data\\customer_support_tickets.csv', 'row': 0}, page_content="Ticket ID: 1\nCustomer Name: Marisa Obrien\nCustomer Email: carrollallison@example.com\nCustomer Age: 32\nCustomer Gender: Other\nProduct Purchased: GoPro Hero\nDate of Purchase: 2021-03-22\nTicket Type: Technical issue\nTicket Subject: Product setup\nTicket Description: I'm having an issue with the {product_purchased}. Please assist.\n\nYour billing zip code is: 71701.\n\nWe appreciate that you have requested a website address.\n\nPlease double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.\nTicket Status: Pending Customer Response\nResolution: \nTicket Priority: Critical\nTicket Channel: Social media\nFirst Response Time: 2023-06-01 12:15:36\nTime to Resolution: \nCustomer Satisfaction Rating: "),
 Document(metadata={'source': 'F:\\GraphRAG-LLM\\data\\customer_support_tickets.csv', 'row': 1}, page_content="Tic

In [7]:
# Initialize a text splitter that divides text into chunks of 512 tokens
# with an overlap of 24 tokens to maintain context between chunks.
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)

# Split the first 100 raw documents into smaller chunks using the text splitter.
documents = text_splitter.split_documents(raw_documents[:10])

In [5]:
llm = ChatGroq(temperature=0, model="gemma2-9b-it")

In [8]:
# Initialize a language model (LLM) using ChatGroq with a specified temperature setting.
# "temperature=0" ensures deterministic responses, reducing randomness.
# The model being used is "llama-3.3-70b-versatile".
llm = ChatGroq(temperature=0, model="gemma2-9b-it")

# Create an instance of LLMGraphTransformer, which will process documents
# into a graph-based structure using the LLM.
llm_transformer = LLMGraphTransformer(llm=llm)

# Convert the previously split text documents into graph-based representations.
# This transformation helps in structuring knowledge for better retrieval.
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [9]:
graph_documents[:5]

[GraphDocument(nodes=[Node(id='Marisa Obrien', type='Person', properties={})], relationships=[Relationship(source=Node(id='Marisa Obrien', type='Person', properties={}), target=Node(id='Carrollallison@Example.Com', type='Email', properties={}), type='HAS_EMAIL', properties={}), Relationship(source=Node(id='Marisa Obrien', type='Person', properties={}), target=Node(id='71701', type='Zip_code', properties={}), type='LIVES_IN', properties={}), Relationship(source=Node(id='Marisa Obrien', type='Person', properties={}), target=Node(id='Gopro Hero', type='Product', properties={}), type='PURCHASED', properties={})], source=Document(metadata={'source': 'F:\\GraphRAG-LLM\\data\\customer_support_tickets.csv', 'row': 0}, page_content="Ticket ID: 1\nCustomer Name: Marisa Obrien\nCustomer Email: carrollallison@example.com\nCustomer Age: 32\nCustomer Gender: Other\nProduct Purchased: GoPro Hero\nDate of Purchase: 2021-03-22\nTicket Type: Technical issue\nTicket Subject: Product setup\nTicket Descrip

In [10]:
# Add the graph-based documents to the existing knowledge graph.
graph.add_graph_documents(
    graph_documents,   # The structured graph documents generated from the LLM.
    baseEntityLabel=True,  # Enables labeling of base entities for better organization and retrieval.
    include_source=True  # Retains the original source information for traceability.
)

In [6]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [7]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [8]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [9]:
google_api_key = os.environ.get("GEMINI_API_KEY")


# Use the API key safely
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)

In [42]:
vector_index = Neo4jVector.from_existing_graph(
    embedding=embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [11]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


c:\conda_envs\agent\lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [14]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [18]:
entity_chain.invoke({"question": "'Jessica Rios ?"}).names

['Jessica Rios']

In [19]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [20]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [21]:
print(structured_retriever("email of Marisa Obrien and 'Jessica Rios',??"))

C:\Users\SURESH BEEKHANI\AppData\Local\Temp\ipykernel_4168\3894304667.py:3: LangChainDeprecationWarning: The function `remove_lucene_chars` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the function exists in the :meth:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :meth:`~langchain-neo4j` and import as `from :meth:`~langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars``.
  words = [el for el in remove_lucene_chars(input).split() if el]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n          

Marisa Obrien - PURCHASED -> Gopro Hero
Marisa Obrien - HAS_EMAIL -> Carrollallison@Example.Com
Marisa Obrien - LIVES_IN -> 71701Jessica Rios - PURCHASED -> Lg Smart Tv
Jessica Rios - HAS_EMAIL -> Clarkeashley@Example.Com
Jessica Rios - HAS_AGE -> 42
Jessica Rios - HAS_GENDER -> Female
Jessica Rios - PURCHASED_ON -> 2021-05-22
Jessica Rios - HAS_TICKET_TYPE -> Technical Issue
Jessica Rios - HAS_TICKET_SUBJECT -> Peripheral Compatibility
Jessica Rios - HAS_TICKET_STATUS -> Pending Customer Response
Jessica Rios - HAS_TICKET_PRIORITY -> Critical
Jessica Rios - USED_CHANNEL -> Chat
Jessica Rios - FIRST_RESPONSE_TIME -> 2023-06-01 16:45:38


In [22]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [23]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [24]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [25]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [26]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGroq(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [27]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [28]:
prompt = ChatPromptTemplate.from_template(template)

In [34]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [36]:
chain.invoke({"question":  "email of Marisa Obrien and Jessica Rios?"})

Search query: email of Marisa Obrien and Jessica Rios?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

"Marisa Obrien's email is carrollallison@example.com and Jessica Rios's email is clarkeashley@example.com.  \n"